In [ ]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
for dirname, _, filenames in os.walk("/content/drive/MyDrive/DL project"):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
!pip install dlib

In [ ]:
import dlib
import cv2
import os
import re
import json
from pylab import *
from PIL import Image, ImageChops, ImageEnhance

In [ ]:
os.mkdir('/content/drive/MyDrive/dataset')
os.mkdir('/content/drive/MyDrive/dataset/real')
os.mkdir('/content/drive/MyDrive/dataset/fake')

In [ ]:
train_frame_folder = '/content/drive/MyDrive/DL project/train_sample_videos'
with open(os.path.join(train_frame_folder, 'metadata.json'), 'r') as file:
    data = json.load(file)
list_of_train_data = [f for f in os.listdir(train_frame_folder) if f.endswith('.mp4')]
#detects face using linear regression and svm
detector = dlib.get_frontal_face_detector()
for vid in list_of_train_data:
    count = 0
    cap = cv2.VideoCapture(os.path.join(train_frame_folder, vid)) 
    frameRate = cap.get(5) #frequency at which the images are generated consectively
    while cap.isOpened():
        frameId = cap.get(1) #to get the current frame no.
        ret, frame = cap.read()
        if ret != True:
            break
        if frameId % ((int(frameRate)+1)*1) == 0:
            face_rects, scores, idx = detector.run(frame, 0)
            #retrieving the detection results 
            for i, d in enumerate(face_rects):
                x1 = d.left()
                y1 = d.top()
                x2 = d.right()
                y2 = d.bottom()
                crop_img = frame[y1:y2, x1:x2]
                if data[vid]['label'] == 'REAL':
                    cv2.imwrite('/content/drive/MyDrive/dataset/real/'+vid.split('.')[0]+'_'+str(count)+'.png', cv2.resize(crop_img, (128, 128)))
                elif data[vid]['label'] == 'FAKE':
                    cv2.imwrite('/content/drive/MyDrive/dataset/fake/ '+vid.split('.')[0]+'_'+str(count)+'.png', cv2.resize(crop_img, (128, 128)))
                count+=1
                

In [ ]:
import os
import cv2
import json
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sn
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

In [ ]:
input_shape = (128, 128, 3)
data_dir = '/content/drive/MyDrive/dataset/'

real_data = [f for f in os.listdir(data_dir+'/real') if f.endswith('.png')]
fake_data = [f for f in os.listdir(data_dir+'/fake') if f.endswith('.png')]

X = []
Y = []
#dividing and assigning data to X and Y variables where X holds the real and fake images and Y is the 1 for real image and 0 for fake image
for img in real_data:
    X.append(img_to_array(load_img(data_dir+'/real/'+img)).flatten() / 255.0)
    Y.append(1)
for img in fake_data:
    X.append(img_to_array(load_img(data_dir+'/fake/'+img)).flatten() / 255.0)
    Y.append(0)

Y_val_org = Y

#Normalization
X = np.array(X)
Y = to_categorical(Y, 2) #converting vector to binary class matrix with 0 and 1 values 

#Reshape
X = X.reshape(-1, 128, 128, 3)

#Train-Test split
X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size = 0.2, random_state=5)

In [ ]:
print(Y_train)
print(X_train)

In [ ]:
from tensorflow.keras.applications import InceptionResNetV2
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import InputLayer
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import Model #takes the input which is an image and output is segmentation class
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
#pretrained model
#loading the weights therefore using imagenet and not including the fully connected layer
googleNet_model = InceptionResNetV2(include_top=False, weights='imagenet', input_shape=input_shape)
googleNet_model.trainable = True
model = Sequential()
model.add(googleNet_model)
model.add(GlobalAveragePooling2D())
model.add(Dense(units=2, activation='softmax'))
# binary_crossentropy function computes the cross-entropy loss 
# between true labels and predicted labels.
model.compile(loss='binary_crossentropy',
              optimizer=optimizers.Adam(lr=1e-5, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False),
              metrics=['accuracy'])
model.summary()

In [ ]:
early_stopping = EarlyStopping(monitor='val_loss',
                               min_delta=0,
                               patience=2,
                               verbose=0, mode='auto')
EPOCHS = 25
BATCH_SIZE = 100
history = model.fit(X_train, Y_train, batch_size = BATCH_SIZE, epochs = EPOCHS, validation_data = (X_val, Y_val), verbose = 1)

In [ ]:
f, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 4))
t = f.suptitle('Pre-trained InceptionResNetV2 Transfer Learn with Fine-Tuning & Image Augmentation Performance ', fontsize=12)
f.subplots_adjust(top=0.85, wspace=0.3)

epoch_list = list(range(1,EPOCHS+1))
ax1.plot(epoch_list, history.history['accuracy'], label='Train Accuracy')
ax1.plot(epoch_list, history.history['val_accuracy'], label='Validation Accuracy')
ax1.set_xticks(np.arange(0, EPOCHS+1, 1))
ax1.set_ylabel('Accuracy Value')
ax1.set_xlabel('Epoch #')
ax1.set_title('Accuracy')
l1 = ax1.legend(loc="best")

ax2.plot(epoch_list, history.history['loss'], label='Train Loss')
ax2.plot(epoch_list, history.history['val_loss'], label='Validation Loss')
ax2.set_xticks(np.arange(0, EPOCHS+1, 1))
ax2.set_ylabel('Loss Value')
ax2.set_xlabel('Epoch #')
ax2.set_title('Loss')
l2 = ax2.legend(loc="best")

In [ ]:
from sklearn.metrics import f1_score

In [ ]:
def print_confusion_matrix(y_true, y_pred):
    cm = confusion_matrix(y_true, y_pred)
    print('True positive = ', cm[0][0])
    print('False positive = ', cm[0][1])
    print('False negative = ', cm[1][0])
    print('True negative = ', cm[1][1])
    print('\n')
    df_cm = pd.DataFrame(cm, range(2), range(2))
    sn.set(font_scale=1.4) # for label size
    sn.heatmap(df_cm, annot=True, annot_kws={"size": 16}) # font size
    plt.ylabel('Actual label', size = 20)
    plt.xlabel('Predicted label', size = 20)
    plt.xticks(np.arange(2), ['Fake', 'Real'], size = 16)
    plt.yticks(np.arange(2), ['Fake', 'Real'], size = 16)
    plt.ylim([2, 0])
    plt.show()

print_confusion_matrix(Y_val_org, np.argmax(model.predict(X), axis=-1))


In [ ]:
model.save('deepfake-detection-model.h5')

In [ ]:
import tensorflow as tf
import dlib
import cv2
import os
import numpy as np
from PIL import Image, ImageChops, ImageEnhance
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import img_to_array, load_img

In [ ]:
tf.__version__

In [ ]:
model = load_model('deepfake-detection-model.h5')

In [ ]:
input_shape = (128, 128, 3)
pr_data = []
detector = dlib.get_frontal_face_detector()
cap = cv2.VideoCapture('/content/drive/MyDrive/DL project/test_videos')
frameRate = cap.get(5)
while cap.isOpened():
    frameId = cap.get(1)
    ret, frame = cap.read()
    if ret != True:
        break
    if frameId % ((int(frameRate)+1)*1) == 0:
        face_rects, scores, idx = detector.run(frame, 0)
        for i, d in enumerate(face_rects):
            x1 = d.left()
            y1 = d.top()
            x2 = d.right()
            y2 = d.bottom()
            crop_img = frame[y1:y2, x1:x2]
            data = img_to_array(cv2.resize(crop_img, (128, 128))).flatten() / 255.0
            data = data.reshape(-1, 128, 128, 3)
            print(model.predict_classes(data))


In [ ]:
import sklearn.metrics
cm = sklearn.metrics.accuracy_score(Y_val_org, np.argmax(model.predict(X), axis=-1))

In [ ]:
print(cm)

In [ ]:

!pip install keras_efficientnets

from keras_efficientnets import EfficientNetB0

In [ ]:
import h5py

In [ ]:
filename = "./deepfake-detection-model.h5"

h5 = h5py.File(filename,'r')
print(h5)
h5.close()

In [ ]:
import keras,os
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPool2D , Flatten
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
from keras.applications.vgg16 import VGG16
from tensorflow.keras.optimizers import SGD,Adam

In [ ]:
base_model = VGG16(include_top = False, weights = 'imagenet', input_shape = input_shape, classes = 2) 
model2= Sequential() 
model2.add(base_model) 
model2.add(Flatten()) 
model2.summary()

In [ ]:
model2.add(Dense(1024,activation=('relu'),input_dim=2))
model2.add(Dense(512,activation=('relu'))) 
model2.add(Dense(256,activation=('relu'))) 

model2.add(Dense(128,activation=('relu')))
#model.add(Dropout(.2))
model2.add(Dense(64,activation=('relu')))
model2.add(Dense(32,activation=('relu')))
model2.add(Dense(8,activation=('relu'))) 
#model.add(Dropout(.4))
model2.add(Dense(2,activation=('softmax'))) 

#Checking the final model summary
model2.summary()

In [ ]:
#try with adam optimizer also 
batch_size= 200
epochs=20 
learn_rate=.001 
sgd=SGD(learning_rate=learn_rate,momentum=.9,nesterov=False) 
adam=Adam(learning_rate=learn_rate, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False) 
model2.compile(optimizer=sgd,loss='binary_crossentropy',metrics=['accuracy'])

In [ ]:
early_stopping = EarlyStopping(monitor='val_loss',
                               min_delta=0,
                               patience=2,
                               verbose=0, mode='auto')
EPOCHS = 25
BATCH_SIZE = 100
history = model.fit(X_train, Y_train, batch_size = BATCH_SIZE, epochs = EPOCHS, validation_data = (X_val, Y_val), verbose = 1)

In [ ]:
# history = model2.fit(X_train,Y_train, batch_size = 100, epochs = 30, validation_data = (X_val,Y_val), verbose = 1)

In [ ]:
def print_confusion_matrix(y_true, y_pred):
    cm = confusion_matrix(y_true, y_pred)
    print('True positive = ', cm[0][0])
    print('False positive = ', cm[0][1])
    print('False negative = ', cm[1][0])
    print('True negative = ', cm[1][1])
    print('\n')
    df_cm = pd.DataFrame(cm, range(2), range(2))
    sn.set(font_scale=1.4) # for label size
    sn.heatmap(df_cm, annot=True, annot_kws={"size": 16}) # font size
    plt.ylabel('Actual label', size = 20)
    plt.xlabel('Predicted label', size = 20)
    plt.xticks(np.arange(2), ['Fake', 'Real'], size = 16)
    plt.yticks(np.arange(2), ['Fake', 'Real'], size = 16)
    plt.ylim([2, 0])
    plt.show()
print_confusion_matrix(Y_val_org, np.argmax(model2.predict(X), axis=-1))



In [ ]:
import sklearn.metrics
cm = sklearn.metrics.accuracy_score(Y_val_org, np.argmax(model2.predict(X), axis=-1))
print(cm)

In [ ]:
!pip install streamlit==0.80.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install pyngrok==4.1.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for pyngrok: filename=pyngrok-4.1.1-py3-none-any.whl size=15983 sha256=a1b03e7e54cfa12236652751956ff1f2ba3db700c2edd3858a05be83c9e0c744
  Stored in directory: /root/.cache/pip/wheels/b1/d9/12/045a042fee3127dc40ba6f5df2798aa2df38c414bf533ca765
Successfully built pyngrok


In [ ]:
%%writefile deepfake_detection_webapp.py 

import streamlit as str
str.header("DEEPFAKE DETECTION")
model=tf.keras.models.load_model("deepfake-detection-model.h5")
user_input=str.write("PLease upload your file here")            //user input
upload_file=str.uploaded_file("Drag and Drop",type="png")     
  
  predicted_output=model.predict(upload_file)          
 if str.button('predict'):                 //creating a button
     if (predicted_output[0]==0):
         str.title("Real")
    else:
         str.title("Fake")

Writing deepfake_detection_webapp.py


In [ ]:
from pyngrok import ngrok

In [ ]:
!nohup streamlit run deepfake_detection_webapp.py & # running streamlit 
public_url = ngrok.connect(port = '8501') # Creating public URL
public_url

nohup: appending output to 'nohup.out'


'http://aa7e-34-143-149-159.ngrok.io'